# init

In [1]:
import sys

sys.path.append('../../')
sys.path.append('../infras/cellMix/')
sys.path.append('../infras/cytof_data/')
sys.path.append('../infras/')
sys.path.append('../infras/dashboards/')
sys.path.append('../experiments/')
sys.path.append('../experiments/pipeline/')
sys.path.append('../preprocess/intra_variance/')
sys.path.append('../models/cell_proportions/')
sys.path.append('../measures/cell_proportions_measures/')
sys.path.append('../preprocess/cell_specifics/')
%matplotlib
import warnings
warnings.filterwarnings('ignore')

Using matplotlib backend: Qt5Agg


In [2]:
from data_factory import DataFactory
from global_utils import GlobalUtils
from cytof_cell_count_infra import CytofCellCountInfra
from cell_proportions_experiments import  CellProportionsExperiments
import exploration_cytof_plots as cytof_plots
from basic import BasicDeconv
from pp_entropy_based import PpEntropyBased
from pp_dep_de_based import  PpDepDeBased
from cell_proportions_measure import CellProportionsMeasure
from pp_clean_high_intra_var import PpCleanHighIntraVar
from pp_clean_irrelevant_proteins import PpCleanIrrelevantProteins
from pp_empty import PpEmpty
from pp_entropy_based_only_largest import PpEntropyBasedOnlyLargest
from aggregate_intra_variance import AggregateIntraVariance
from pipeline_deconv import PipelineDeconv
from deconv_py.preprocess.base import BasePreprocess as PP_base
from deconv_py.preprocess.cell_specific import CellSpecific as PP_proteins

from pp_keep_specific_cells  import  PpKeepSpecificCells
from pp_agg_to_specific_cells import PpAggToSpecificCells

# from deconv_py.infras.data_factory import DataFactory
from deconv_py.infras.data_loader import DataLoader
from deconv_py.models.base import Base as Models_base
from deconv_py.models.cell_proportions_models import CellProportions
from deconv_py.models.cell_specific_models import CellSpecificPerPermutation
from deconv_py.experiments.cell_specific import CellSpecificMetricsPlot
from cellMix_coordinator import CellMixCoordinator

import pandas as pd
import numpy as np
from functools import partial
import multiprocessing
from sklearn import pipeline
import itertools
from scipy.optimize import least_squares
from sklearn.metrics import mean_squared_error
from functools import partial
from scipy.optimize import minimize
import scipy.optimize
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import os
import pickle as pkl 
from sklearn.decomposition import PCA
from IPython.display import display, HTML

from itertools import chain
from scipy.optimize import least_squares
from sklearn.metrics import mean_squared_error
from functools import partial
from scipy.optimize import minimize
import scipy.optimize
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
import math


from scipy.stats import entropy
from functools import reduce
import matplotlib.patches as patches


import os


%connect_info

{
  "shell_port": 53710,
  "iopub_port": 53711,
  "stdin_port": 53712,
  "control_port": 53713,
  "hb_port": 53714,
  "ip": "127.0.0.1",
  "key": "b8424777-8571909b0fe69ac12df028fb",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-9d991327-7827-4ae5-bfe2-9053447b0f1d.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


#  read data

In [3]:
#old data - imputed
data_factory = DataFactory()
A_org,B_org = data_factory.load_IBD_all_vs("Intensity",index_func=lambda x:x.split(";")[0],log2_transformation=True)

# #old data - not imputed
data_factory = DataFactory()
A_org_not_imp,B_org_not_imp = data_factory.load_no_imputation_IBD_all_vs("Intensity",index_func=lambda x:x.split(";")[0],log2_transformation=False)
B_org_not_imp = B_org_not_imp.replace('Filtered',0)
B_org_not_imp=B_org_not_imp.astype(float)

#new data - not targeted
data_factory = DataFactory()
A_new,B_new = data_factory.load_2020_IBD_all_vs("Intensity",index_func=lambda x:x.split(";")[0],log2_transformation=False)
B_new = B_new.replace('Filtered',0)

#new data - targeted
data_factory = DataFactory()
A_targeted,B_targeted = data_factory.load_2020_targeted_IBD_all_vs("Intensity",index_func=lambda x:x.split(";")[0],log2_transformation=False)
B_targeted = B_targeted.replace('Filtered',0)

In [4]:
raise

RuntimeError: No active exception to reraise

# code base

In [ ]:
def plot_expcted_mixtures_vs_real_mixtures(expcted_mixtures,real_mixtures,xlabel,ylabel,lims=None,percantile_to_take = 1,as_log=True,correlation_type =  "spearman"):
    mutual_mixtures = expcted_mixtures.columns.intersection(real_mixtures.columns)
    n_figs = len(mutual_mixtures)
    fig, axs = plt.subplots(math.ceil(n_figs/2),2,figsize=(10,10))

    for mixture,ax in zip(mutual_mixtures,axs.reshape(-1)) :
        
        expcted_mixture_expression = expcted_mixtures[mixture]
        real_mixture_expression = real_mixtures[mixture]
        
        exp_robust = expcted_mixtures[expcted_mixtures <= expcted_mixtures.quantile(percantile_to_take)].dropna().index
        B_robust = real_mixtures[real_mixtures <= real_mixtures.quantile(percantile_to_take)].dropna().index
        mutual_robust = exp_robust.intersection(B_robust)
        
        expcted_mixture_expression = expcted_mixture_expression.loc[mutual_robust]
        real_mixture_expression = real_mixture_expression.loc[mutual_robust]
        
        if as_log : 
            x = np.log(1+expcted_mixture_expression)
            y = np.log(1+real_mixture_expression)
        else : 
            x = expcted_mixture_expression
            y = real_mixture_expression

        correlation_with_out_zeros = np.round(real_mixture_expression[x != 0].corr(expcted_mixture_expression[x != 0],method=correlation_type),3)
        xy = np.vstack([x,y])
        z = gaussian_kde(xy)(xy)
        
        
       
        ax.scatter(x.values, y.values, c=z,s=100)
        ax.set_title(f"{mixture}, {correlation_type} of non zeros(real not log) : {correlation_with_out_zeros}")
        ax.set_xlabel(f"{xlabel} log(1+x)")
        ax.set_ylabel(f"{ylabel} log(1+x)")
        
        if lims is not None : 
            ax.set_xlim(lims[0])
            ax.set_ylim(lims[1])

    plt.subplots_adjust(hspace=1)  

def plot_ranked_expcted_mixtures_vs_real_mixtures(expcted_mixtures,real_mixtures,xlabel,ylabel,lims=None,percantile_to_take = 1,as_log=True,correlation_type =  "spearman"):
    mutual_mixtures = expcted_mixtures.columns.intersection(real_mixtures.columns)
    n_figs = len(mutual_mixtures)
    fig, axs = plt.subplots(math.ceil(n_figs/2),2,figsize=(10,10))

    for mixture,ax in zip(mutual_mixtures,axs.reshape(-1)) :
        
        expcted_mixture_expression = expcted_mixtures[mixture].rank()
        real_mixture_expression = real_mixtures[mixture].rank()
        
        exp_robust = expcted_mixtures[expcted_mixtures <= expcted_mixtures.quantile(percantile_to_take)].dropna().index
        B_robust = real_mixtures[real_mixtures <= real_mixtures.quantile(percantile_to_take)].dropna().index
        mutual_robust = exp_robust.intersection(B_robust)
        
        expcted_mixture_expression = expcted_mixture_expression.loc[mutual_robust]
        real_mixture_expression = real_mixture_expression.loc[mutual_robust]
        
        if as_log : 
            x = np.log(1+expcted_mixture_expression)
            y = np.log(1+real_mixture_expression)
        else : 
            x = expcted_mixture_expression
            y = real_mixture_expression

        correlation_with_out_zeros = np.round(real_mixture_expression[x != 0].corr(expcted_mixture_expression[x != 0],method=correlation_type),3)
        xy = np.vstack([x,y])
        z = gaussian_kde(xy)(xy)
        
        
       
        ax.scatter(x.values, y.values, c=z,s=100)
        ax.set_title(f"{mixture}, {correlation_type} of non zeros(real not log) : {correlation_with_out_zeros}")
        ax.set_xlabel(f"{xlabel} log(1+x)")
        ax.set_ylabel(f"{ylabel} log(1+x)")
        
        if lims is not None : 
            ax.set_xlim(lims[0])
            ax.set_ylim(lims[1])

    plt.subplots_adjust(hspace=1)  
    
def corr_expcted_vs_mixture(expcted_b,B,xlabel,ylabel,method  = "pearson",n_to_take = 10):
    percs = np.linspace(0,100,21)
    mutual_mixtures = B.columns.intersection(expcted_b.columns)
    n_to_take = min(n_to_take,len(mutual_mixtures))
    corr_per_mixture_df = pd.DataFrame(index= mutual_mixtures)

    for i in np.linspace(0.45,0.95,10) :
        trh = round(i,2)

        for mixture_idx in mutual_mixtures:
            mixture = B[mixture_idx]
            mixture_trh = mixture.quantile(trh)

            _expcted_b = expcted_b[mixture_idx]
            _expcted_b_trh = _expcted_b.quantile(trh)
            
            relevant_proteins = mixture[mixture>mixture_trh].index.intersection(_expcted_b[_expcted_b>_expcted_b_trh].index)
            
            corr = _expcted_b.loc[relevant_proteins].corr(mixture.loc[relevant_proteins],method = method)
            corr_per_mixture_df.ix[mixture_idx,trh] = corr
            
    ax = corr_per_mixture_df.T.sample(n=n_to_take,axis=1).plot(title=f"{xlabel} vs {ylabel} - {method}")
    ax.set_xlabel("percantile")
    ax.set_ylabel(method)
    plt.show()

def plot_diff_expcted_mixtures_vs_real_mixtures(expcted_mixtures,real_mixtures,title = ""):
    mutual_mixtures = expcted_mixtures.columns.intersection(real_mixtures.columns)

    for mixture in mutual_mixtures :
        expcted_mixture_expression = expcted_mixtures[mixture]
        real_mixture_expression = real_mixtures[mixture]
        mutual = expcted_mixture_expression.index.intersection(real_mixture_expression.index)

        expcted_mixture_expression = expcted_mixture_expression.loc[mutual]
        real_mixture_expression = real_mixture_expression.loc[mutual]

        _mean = (expcted_mixture_expression + real_mixture_expression)/2
        _diff = abs(expcted_mixture_expression - real_mixture_expression)
        _diff = _diff.fillna(0)
        
        x=np.log(1+_mean)
        y =np.log(1+ _diff)

        xy = np.vstack([x,y])
        z = gaussian_kde(xy)(xy)

        fig,ax = plt.subplots()
        ax.scatter(x.values, y.values, c=z,s=100)
        ax.set_title(f"{title} - {mixture}")
        ax.set_xlabel(f"mean")
        ax.set_ylabel(f"diff")
        plt.show()

    plt.subplots_adjust(hspace=1) 


# comparison 

## values heatmaps : log(1+x) vs log(1+y)

### original vs new

In [ ]:
plot_expcted_mixtures_vs_real_mixtures(B_new,B_org,"new","original")

### not_imputed vs new

In [ ]:
plot_expcted_mixtures_vs_real_mixtures(B_new,B_org_not_imp,"new","not_imputed")

### new vs targeted

In [ ]:
plot_expcted_mixtures_vs_real_mixtures(B_targeted,B_new,"targeted","new",lims=None)

### original vs targeted

In [ ]:
plot_expcted_mixtures_vs_real_mixtures(B_targeted,B_org,"targeted","original")

## ranking  heatmaps : rank(x) vs rank(y)

### original vs new

In [ ]:
plot_ranked_expcted_mixtures_vs_real_mixtures(B_new,B_org,"new","original",as_log=False)

### not_imputed vs new

In [ ]:
plot_ranked_expcted_mixtures_vs_real_mixtures(B_new,B_org_not_imp,"new","not_imputed")

### new vs targeted

In [ ]:
plot_ranked_expcted_mixtures_vs_real_mixtures(B_targeted,B_new,"targeted","new",lims=None,as_log=False)

### original vs targeted

In [ ]:
plot_ranked_expcted_mixtures_vs_real_mixtures(B_targeted,B_org,"targeted","original",as_log=False)

## corr per percentile

### original vs new

In [ ]:
corr_expcted_vs_mixture(B_org,B_new,"original","new",method="spearman")

### not_imputed vs new

In [ ]:
corr_expcted_vs_mixture(B_new,B_org_not_imp,"new","not_imputed")

### new vs targeted

In [ ]:
corr_expcted_vs_mixture(B_targeted,B_new,"targeted","new",method="spearman")

### original vs targeted

In [ ]:
corr_expcted_vs_mixture(B_targeted,B_org,"targeted","original",method="spearman")

## diff : log(1+abs(x-y)) vs log( 1+ mean(x,y))

### original vs new

In [ ]:
plot_diff_expcted_mixtures_vs_real_mixtures(B_org,B_new,"original - new")

### not_imputed vs new

In [ ]:
plot_diff_expcted_mixtures_vs_real_mixtures(B_new,B_org_not_imp,"new-not_imputed")

### new vs targeted

In [ ]:
plot_diff_expcted_mixtures_vs_real_mixtures(B_targeted,B_new,"targeted - new")

### original vs targeted

In [ ]:
plot_diff_expcted_mixtures_vs_real_mixtures(B_targeted,B_org,"targeted-original")

# questions

# moving forward 